# Mathematik für Biologiestudierende II

Sommersemester 2025

24.06.2025

&copy; 2025 Prof. Dr. Rüdiger W. Braun 

# Themen



In [ ]:
import numpy as np
np.set_printoptions(legacy='1.21')
import pandas as pd
from scipy import stats
import seaborn as sns
sns.set_theme()
import statsmodels.formula.api as smf

# Multikollinearität

* eine Menge von Größen ist *kollinear*, wenn sich eine von ihnen linear aus den anderen bestimmen lässt
* "linear" heißt durch Addition und Multiplikation mit Skalaren 

In Lektion 25 gesehen:

> midparentHeight = (father + 1.08 * mother)/2

* also sind `father`, `mother` und `childHeight` kollinear
* Multikollinearität im statistischen Sinne liegt bereits dann vor, wenn eine Gleichung wie die obige fast zutrifft

* eine Maßzahl für Multikollinearität ist die Konditionszahl

# Konditionszahl

In [ ]:
df = pd.read_csv('galton.csv')

In [ ]:
formel = "childHeight ~ father + mother"
model = smf.ols(formel, df)

In [ ]:
res = model.fit()

In [ ]:
res.summary()

* `condition number` bedeutet Konditionszahl

* Um das Modell zu berechnen, muss ein lineares Gleichungssystem gelöst werden
* Je größer die Konditionszahl, umso schwieriger ist die Berechnung der Lösung
* Für den Nutzer bedeutet das, dass die ausgegebenen Werte mit Unsicherheiten behaftet sind
* und im Extremfall sogar falsch

* eine Konditionszahl von 4000 ist aber noch nicht besorgniserregend groß

#### Ein nicht statistisches Beispiel mit schlechter Konditionszahl

Wir betrachten das Gleichungssystem
$$
    \begin{aligned}
        x + y &= 1 \\
        x + 1.0000001 y &= 1.001
    \end{aligned}
$$


Die Lösung ist
$$
    x = -9999, y = 10000
$$

* Die Konditionszahl des Gleichungssystems kann mit `numpy` ausgerechnet werden
* Sie beträgt 40000001.9

* Modelle mit schechter Konditionszahl müssen vermieden werden
* Wenn das Gleichungssystem aus dem Beispiel aus der Biologie käme, dann wären die Zahlen Messwerte
* also mit Unsicherheiten behaftet

* In einem System mit schlechter Konditionszahl führen winzige Messungenauigkeiten bereits zu völlig unsinnigen Ergebnissen

#### Beispiel mit wirklich schlechter Kondition

In [ ]:
formel = "childHeight ~ father + mother + midparentHeight"

In [ ]:
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

> midparentHeight = (father + 1.08 * mother)/2

* Daher kann man aus diesen drei Größen beliebig viele unterschiedliche Modelle bauen, die alle dasselbe vorhersagen
* Andererseits ist das Modell enorm empfindlich gegenüber Änderungen der Ausgangsdaten

* Wir ändern die Daten um ein winziges bisschen
* Wir machen alle Mütter um 3µm kleiner

In [ ]:
df['m1'] = df.mother - 0.0001

In [ ]:
f1 = "childHeight ~ m1 + father + midparentHeight"
m1 = smf.ols(f1, df)
r1 = m1.fit()

In [ ]:
r1.summary()

Ein offenkundig sinnloses Modell

Das Problem ist, dass Multikollinearität in der Praxis häufig verborgen ist

# Kategorielle Daten

In [ ]:
df = pd.read_csv('kinder.csv')
df.head()

In [ ]:
sns.lmplot(df, x='father', y='childHeight', hue='gender');

In [ ]:
formel = 'childHeight ~ father + mother + gender'
modell = smf.ols(formel, df)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

Hier wird eine Fallunterscheidung kodiert

$$
    \text{childHeight} = 16.5212 + 0.3928 \cdot \text{father} + 0.13176 \cdot \text{mother} + 
    \begin{cases}
        0.0 & \text{wenn Mädchen,} \\
        5.215 & \text{wenn Junge.}
    \end{cases}
$$

Die Terminologie kommt offenbar aus der Pharmazie:

* `female` ist hier der Standard (engl. "default")
* alles, was vom Standard abweicht, ist eine Behandlung (engl. "treatment")
* was default und was treatment ist, entscheidet das Programm

## Prediction im kategoriellen Fall

Beispiel aus Lektion 25

In [ ]:
anfrage = pd.DataFrame()
anfrage['father'] = [68.9]
anfrage['mother'] = [66.4]
anfrage['gender'] = ['male']
anfrage

In [ ]:
res.get_prediction(anfrage).summary_frame()

* In Lektion 25 war 69.91 herausgekommen
* Dies ist kein Rundungsfehler
* Es wird eine gemeinsame Steigung für alle Kinder berechnet